# Freshwater/Salt Fluxes through Bering Strait and the Strait of Gibraltar
Med salt flux can be checked with net SFWF virtual salt flux

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import pickle
import xarray as xr
import cmocean
import cartopy
import warnings  
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import notebook
from paths import file_ex_ocn_ctrl, file_ex_ocn_lpd
from paths import path_results, path_prace, file_RMASK_ocn, file_RMASK_ocn_low
from FW_plots import FW_region_plot, FW_summary_plot
from FW_budget import load_obj, lat_bands
from constants import rho_sw
from timeseries import IterateOutputCESM
from FW_transport import calc_section_transport, sections_high, sections_low
from xr_DataArrays import xr_AREA, xr_DZ
from xr_regression import ocn_field_regression, xr_linear_trend
warnings.filterwarnings('ignore')

In [ ]:
dsh = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
dsl = xr.open_dataset(file_ex_ocn_lpd , decode_times=False)

### time series of SALT/volume/FW fluxes 

In [ ]:
ds_ctrl = xr.open_mfdataset(f'{path_prace}/Mov/FW_Med_Bering_ctrl_*.nc', concat_dim='time', decode_times=False)
ds_rcp  = xr.open_mfdataset(f'{path_prace}/Mov/FW_Med_Bering_rcp_*.nc' , concat_dim='time', decode_times=False)
ds_lpd  = xr.open_mfdataset(f'{path_prace}/Mov/FW_Med_Bering_lpd_*.nc' , concat_dim='time', decode_times=False)
ds_lr1  = xr.open_mfdataset(f'{path_prace}/Mov/FW_Med_Bering_lr1_*.nc' , concat_dim='time', decode_times=False)

In [ ]:
# into the Arctic Ocean a a volume flux of 0.8~Sv
# Mediterranean is a net evaporative basin with a net volume inflow of $\SI{0.038(7)}{Sv}$
# assuming outflow of 0.8 Sv of 38.4 psu and inflow of 36.2 psu = -1.76 kt/s
pm = '+/-'
f, ax = plt.subplots(2, 3, figsize=(8,5))
ax[0,0].scatter(1995, 26, c='k')
ax[0,0].errorbar(1995, 26, yerr=3, c='k')
ax[0,1].scatter(1995, 0.8, c='k')
ax[0,1].errorbar(1995, 0.8, yerr=0.1, c='k')

ax[1,0].scatter(1995, -8e5*(38.4-36.2)/1e6, c='k')
ax[1,1].scatter(1995, 0.038, c='k')
ax[1,1].errorbar(1995, 0.038, yerr=0.007, c='k')
for i in range(3):  ax[0,i].set_title(['salt [kt/s]', 'volume [Sv]', 'freshwater [Sv]'][i])
print('run            Bering Strait      Gibraltar')
for j, ds in enumerate([ds_ctrl, ds_rcp, ds_lpd, ds_lr1]):
    run = ['HIGH ctrl', 'HIGH rcp', 'LOW  ctrl', 'LOW  rcp'][j]
    time = ds.time.values/365
    if j==0: time += 1800
    if j==2: time += 1500
    for i, sec in enumerate(['BS', 'Med']):
        if j==0: ax[i,0].set_ylabel(['Bering Strait', 'Strait of Gibraltar'][i])
        ax[i,0].plot(time, ds[f'S_{sec}']/1e6, lw=.5, label=f'{run.upper()}')
        ax[i,1].plot(time, ds[f'V_{sec}']/1e6, lw=.5, label=f'{run.upper()}')
        ax[i,2].plot(time, ds[f'F_{sec}']/1e6, lw=.5, label=f'{run.upper()}')
    for q in ['S', 'V', 'F']:
        mBS  = ds[f'{q}_BS'].mean().values/1e6
        sBS  = ds[f'{q}_BS'].std().values/1e6
        mMed = ds[f'{q}_Med'].mean().values/1e6
        sMed = ds[f'{q}_Med'].std().values/1e6
        print(f'{run:10} {q}   {mBS:6.3f} {pm} {sBS:5.3f}   {mMed:7.4f} {pm} {sMed:6.4f}')
ax[1,1].legend(fontsize=8)
f.align_ylabels()
plt.savefig(f'{path_results}/Mov/BS_Med_fluxes')

In [ ]:
from xr_regression import xr_linear_trend, xr_lintrend

In [ ]:
??xr_linear_trend
??xr_lintrend

### geometry of sections

In [ ]:
f, ax = plt.subplots(2,2, figsize=(12,10))
for i, sim in enumerate(['HIGH', 'LOW']):
    ds = [dsh, dsl][i]
    DZ = xr_DZ(['ocn','ocn_low'][i])
    d = [sections_high, sections_low][i]
    
    ds.UES.isel(d['Med']).plot(ax=ax[0,i])
    ax[0,i].set_ylim((0,1e5))
    ax[0,i].set_title(f'{sim} Med     {(ds.UES*DZ*ds.TAREA).isel(d["Med"]).sum().values/1e4:4.2e} kg/s')
    
    ds.VNS.isel(d['Bering']).plot(ax=ax[1,i])
    ax[1,i].set_ylim((0,6e3))
    ax[1,i].set_title(f'{sim} Bering  {(ds.VNS*DZ*ds.TAREA).isel(d["Bering"]).sum().values/1e4:4.2e} kg/s')

In [ ]:
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(satellite_height=1e6, central_latitude=65, central_longitude=190, globe=None))
ax.coastlines(resolution='110m')
plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-100, vmax=100)
ax.gridlines()
plt.colorbar()

In [ ]:
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=65, central_longitude=190))
ax.set_extent((170,210,51,75), crs = ccrs.PlateCarree())
ax.coastlines(resolution='110m')
plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-100, vmax=100)
ax.gridlines()
plt.colorbar()

In [ ]:
def plot_nlat_nlon_boxes(section, ds, ax=None):
    """ draw boundaries around sections """
    assert type(section)==dict
    for q in ['TLONG', 'TLAT', 'ULONG', 'ULAT']:
        assert q in ds
    if ax==None:  ax = plt.gca()
    if type(section['nlon'])==int and type(section['nlat'])==slice:  # meridional section
        nlon = section['nlon']
        nlat1 = section['nlat'].start
        nlat2 = section['nlat'].stop
        
        lats1 = ds.ULAT .isel(nlon=nlon-1).isel(nlat=slice(nlat1-1,nlat2)).values
        lons1 = ds.ULONG.isel(nlon=nlon-1).isel(nlat=slice(nlat1-1,nlat2)).values
        lats2 = ds.ULAT .isel(nlon=nlon).isel(nlat=slice(nlat1-1,nlat2)).values
        lons2 = ds.ULONG.isel(nlon=nlon).isel(nlat=slice(nlat1-1,nlat2)).values
        ax.plot(lons1, lats1, transform=ccrs.PlateCarree(), c='C1')
        ax.plot(lons2, lats2, transform=ccrs.PlateCarree(), c='C1')
        
        lats1 = ds.TLAT .isel(nlon=nlon).isel(nlat=slice(nlat1,nlat2+1)).values
        lons1 = ds.TLONG.isel(nlon=nlon).isel(nlat=slice(nlat1,nlat2+1)).values
        lats2 = ds.TLAT .isel(nlon=nlon+1).isel(nlat=slice(nlat1,nlat2+1)).values
        lons2 = ds.TLONG.isel(nlon=nlon+1).isel(nlat=slice(nlat1,nlat2+1)).values
        ax.plot(lons1, lats1, transform=ccrs.PlateCarree(), c='C2')
        ax.plot(lons2, lats2, transform=ccrs.PlateCarree(), c='C2')
    
    elif type(section['nlon'])==slice and type(section['nlat'])==int:  # zonal section
        nlon = section['nlon']
        nlat1 = section['nlat'].start
        nlat2 = section['nlat'].stop
        
        lats1 = ds.ULAT .isel(nlon=nlon-1).isel(nlat=slice(nlat1-1,nlat2)).values
        lons1 = ds.ULONG.isel(nlon=nlon-1).isel(nlat=slice(nlat1-1,nlat2)).values
        lats2 = ds.ULAT .isel(nlon=nlon).isel(nlat=slice(nlat1-1,nlat2)).values
        lons2 = ds.ULONG.isel(nlon=nlon).isel(nlat=slice(nlat1-1,nlat2)).values
        ax.plot(lons1, lats1, transform=ccrs.PlateCarree(), c='C1')
        ax.plot(lons2, lats2, transform=ccrs.PlateCarree(), c='C1')
        
        lats1 = ds.TLAT .isel(nlon=nlon).isel(nlat=slice(nlat1,nlat2+1)).values
        lons1 = ds.TLONG.isel(nlon=nlon).isel(nlat=slice(nlat1,nlat2+1)).values
        lats2 = ds.TLAT .isel(nlon=nlon+1).isel(nlat=slice(nlat1,nlat2+1)).values
        lons2 = ds.TLONG.isel(nlon=nlon+1).isel(nlat=slice(nlat1,nlat2+1)).values
        ax.plot(lons1, lats1, transform=ccrs.PlateCarree(), c='C2')
        ax.plot(lons2, lats2, transform=ccrs.PlateCarree(), c='C2')
        print('')
    else:
        print('not the right section type')
    return

In [ ]:
f = plt.figure(figsize=(8, 5))

for i, sim in enumerate(['HIGH', 'LOW']):
    ds = [dsh, dsl][i]
    d = [sections_high, sections_low][i]
    ax = f.add_subplot(1, 2, i+1, projection=ccrs.PlateCarree())
    ax.set_extent((-10,5,32,40), crs=ccrs.PlateCarree())
    ax.set_position([.03+i*.5,.1,.44,.8])

    ax.coastlines(resolution='50m')
    ax.pcolormesh(ds.TLONG, ds.TLAT, ds.SSH.where(ds.REGION_MASK>0).squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-100, vmax=100)
    ax.gridlines()
    plot_nlat_nlon_boxes(d['Med'], ds, ax=ax)

In [ ]:
f = plt.figure(figsize=(8, 5))

for i, sim in enumerate(['HIGH', 'LOW']):
#     if i==1: continue
    ds = [dsh, dsl][i]
    ax = f.add_subplot(1, 2, i+1, projection=ccrs.PlateCarree())
    ax.set_extent((-7,-5,35.5,36.5), crs=ccrs.PlateCarree())
    ax.set_position([.03+i*.5,.1,.44,.8])

    ax.coastlines(resolution='50m')
    ax.pcolormesh(ds.ULONG, ds.ULAT, ds.VVEL.isel(z_t=0).where(ds.REGION_MASK>0).squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-100, vmax=100)
    ax.gridlines()
    if i==0:
        plot_nlat_nlon_boxes(hdict['Med'], dsh, ax=ax)
        plot_nlat_nlon_boxes({'nlon':1042, 'nlat':slice(1555,1559)}, dsh, ax=ax)
    if i==1:
        plot_nlat_nlon_boxes(ldict['Med'], dsl, ax=ax)
        ax.scatter(lons, lats, transform=ccrs.PlateCarree(), c='k')
        plot_nlat_nlon_boxes({'nlon':32, 'nlat':slice(290,295)}, dsl, ax=ax)

In [ ]:
dsh.SSH

In [ ]:
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=35, central_longitude=-5))
ax.set_extent((-10,5,32,40), crs = ccrs.PlateCarree())
ax.coastlines(resolution='50m')
# plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-100, vmax=100)
plt.pcolormesh(dsh.TLONG, dsh.TLAT, dsh.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-100, vmax=100)
ax.gridlines()
plt.colorbar()
plot_nlat_nlon_boxes(ldict['Med'], dsl, ax=ax)

In [ ]:
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=35, central_longitude=-5))
ax.set_extent((-10,3,32,40), crs = ccrs.PlateCarree())
ax.coastlines(resolution='110m')
plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.VVEL.isel(z_t=0).squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance', vmin=-10, vmax=10)
ax.gridlines()
plt.colorbar()

In [ ]:
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(satellite_height=1e6, central_latitude=65, central_longitude=190))
ax.coastlines(resolution='110m')
plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance')
ax.gridlines()
plt.colorbar()

In [ ]:
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(satellite_height=1e6, central_latitude=65, central_longitude=190))
ax.coastlines(resolution='110m')
plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance')
ax.gridlines()
plt.colorbar()
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(satellite_height=1e6, central_latitude=65, central_longitude=190))
ax.coastlines(resolution='110m')
plt.pcolormesh(dsh.TLONG, dsh.TLAT, dsh.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance')
ax.gridlines()
plt.colorbar()

In [ ]:
proj = ccrs.PlateCarree(central_longitude=190)
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=proj)
ax.set_extent(())
ax.coastlines(resolution='110m')
plt.pcolormesh(dsl.TLONG, dsl.TLAT, dsl.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance')
ax.gridlines()
plt.colorbar()
plt.figure(figsize=(4, 3))
ax = plt.axes(projection=proj)
ax.coastlines(resolution='110m')
plt.pcolormesh(dsh.TLONG, dsh.TLAT, dsh.SSH.squeeze(), transform=ccrs.PlateCarree(), cmap='cmo.balance')
ax.gridlines()
plt.colorbar()

In [ ]:
f'{(3e4*3600*24*365)/2.5e12:e} m/yr'

In [ ]:
da_UES  = xr.open_dataset(f'{path_prace}/ctrl/ocn_yrly_UES_0202.nc' , decode_times=False)

In [ ]:
da_UES

In [ ]:
ds_ = ds.isel({'nlon': slice(191, 203, None), 'nlat': slice(335,337), 'z_t':0})

In [ ]:
ds_.SALT.values.flatten()

In [ ]:
ds_.TLAT.values

In [ ]:
im1 = plt.scatter(x=ds_.TLONG.values.flatten(), y=ds_.TLAT.values.flatten(), c=ds_.SALT.values.flatten(), cmap='cmo.haline')
im2 = plt.scatter(x=ds_.ULONG.values.flatten(), y=ds_.ULAT.values.flatten(), c=ds_.VVEL.values.flatten(), cmap='RdBu_r', marker='s', vmin=-40,vmax=40)
plt.colorbar()